In [1]:
import pandas as pd
import csv
import numpy as np
import random
import numpy as np
import networkx as nx
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import pickle

### Loading Train Dataframe

In [13]:
train_df = pd.read_csv("data/20240303_2357_train_directed.csv")
train_df.head(3)

,source,sink,label,source_in_degree_dens,sink_in_degree_dens,source_out_degree_dens,sink_out_degree_dens,source_bi_degree_dens,sink_bi_degree_dens,common_in_neighbours,common_out_neighbours,total_in_neighbours,total_out_neighbours,source_nh_subgraph_dens,sink_nh_subgraph_dens
0,1000879,3408999,1,0.25,0.500000,0.75,0.500000,0.000000,0.000000,0,0,5,13,16.0,2.0
1,3109148,216786,1,0.00,1.000000,1.00,0.000000,0.000000,0.000000,0,0,1,8,8.0,1.0
2,4035827,2217836,1,0.50,0.266667,0.50,0.733333,0.166667,0.066667,0,0,7,14,6.0,15.0


In [14]:
train_df.shape

(47515, 15)

In [15]:
train_df.label.value_counts()

label
1    25000
0    22515
Name: count, dtype: int64

### Model

In [16]:
X = train_df.drop(['source','sink','label'], axis=1)
y = train_df['label']

In [17]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [18]:
model = LogisticRegression(C=1, class_weight = None)

In [19]:
# Specify the number of folds (k)
num_folds = 5

# Create a KFold object
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1)

# Use cross_val_score for cross-validation
cv_results = cross_val_score(model, X, y, cv=kf, scoring='roc_auc')

# Print the cross-validation results
print("Cross-Validation Results:", cv_results)
print("Mean AUC:", cv_results.mean())

Cross-Validation Results: [0.90442003 0.89849247 0.89856367 0.89963036 0.89735592]
Mean AUC: 0.8996924894514768


In [20]:
model.fit(X,y)

LogisticRegression(C=1)

In [21]:
model.coef_

array([[-2.92152167e+00,  4.56662740e-01,  3.37412737e+00,
        -4.05704461e-03,  1.96314034e+01, -1.28075760e-01,
         3.25472211e+00,  4.53546751e+00,  2.55199475e+00,
        -7.13876123e+00, -1.54367050e+01,  1.06651444e+01]])

### Saving the Scaler

In [22]:
today_date = datetime.now().strftime("%Y%m%d_%H%M")
with open(f"data\{today_date}_scaler.pkl", "wb") as f:
    loaded_model = pickle.dump(scaler, f)

### Saving the trained model

In [23]:
with open(f"data\{today_date}_model.pkl", "wb") as f:
    loaded_model = pickle.dump(model, f)